https://chat.deepseek.com/a/chat/s/b693a3a0-dd59-4cc2-b6b7-67484f101d65

In [7]:
from data_loader import load_data
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

In [8]:
df = load_data()
df.head(3)

,tenure,monthly_charges,total_charges,senior_citizen,phone_service,multiple_lines,internet_service,online_security,online_backup,device_protection,...,streaming_tv,streaming_movies,contract,paperless_billing,payment_method,customer_id,churn,gender,partner,dependents
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
